In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math 
import time

In [ ]:
img = mpimg.imread('joconde.jpg')  

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
gray = rgb2gray(img)

In [ ]:
def plot_gray(img):
    plt.imshow(img, cmap='gray')
    plt.show()

In [ ]:
plot_gray(gray)

In [ ]:
def crop_image_circle(image,center,radius):
    gray_tmp = np.copy(image)
    for i in range(gray_tmp.shape[0]):
        for j in range(gray_tmp.shape[1]):
            distance_2 = math.sqrt(pow(center[0]-i,2) + pow(center[1]-j,2))
            if distance_2 > radius:
                gray_tmp[i][j] = 255
    return gray_tmp

In [ ]:
center = [350,470]
radius = 300

In [ ]:
gray_crop = crop_image_circle(gray,center,radius)

In [ ]:
plot_gray(gray_crop)

In [ ]:
gray_crop[200][800]

In [ ]:
def generate_nail_positions(center, radius, number):
    x = [center[0] + radius*math.cos(n*2*math.pi/number) for n in range(number)]
    y = [center[1] + radius*math.sin(n*2*math.pi/number) for n in range(number)]
    res = {}
    for i in range(number):
        res[i] = [int(x[i]),int(y[i])]
    return res

In [ ]:
def generate_line_matrix(line,shape):
    line_matrix = 255*np.ones(shape=(shape[0],shape[1]))
    point1 = line[0]
    point2 = line[1]
    if abs(point2[0]-point1[0]) > 0.001:
        a = (point2[1]-point1[1])/(point2[0]-point1[0])
        b = point1[1]-a*point1[0]
        x = [i for i in range(min(point1[0],point2[0]),max(point1[0],point2[0])+1)]
        y = [round(a*i+b) for i in x]
        for x_ in x:
            y_ = round(a*x_+b)
            line_matrix[x_][y_] = 0
    else:
        if abs(point2[1]-point1[1]) < 0.001:
            print('none')
            return None
        y = [i for i in range(min(point1[1],point2[1]),max(point1[1],point2[1])+1)]
        for y_ in y:
            line_matrix[point1[0]][y_] = 0        
    return line_matrix

In [ ]:
line=[[500,600],[70,150]]
plot_gray(generate_line_matrix(line,gray.shape))

In [ ]:
def add_line(current_idx,next_idx, nail_positions, image, line_dict):
    a = min(current_idx,next_idx)
    b = max(current_idx,next_idx)
    key = str(a)+'_'+str(b)
    if key not in line_dict.keys():
        line = [nail_positions[current_idx],nail_positions[next_idx]]
        line_dict[key] = generate_line_matrix(line,image.shape)
    return np.multiply(image,line_dict[key])

In [ ]:
def distance_image(image1, image2):
    return np.linalg.norm(image1-image2)

In [ ]:
def possible_next_nails(idx, nail_number, threshold = 10):
    # 0 <= idx <= nail_number-1
    if (idx>=nail_number):
        print('pb')
        return []
    if ((idx>=threshold) and (idx+threshold<nail_number)):
        left = [i for i in range(1,idx-threshold)]
        right = [i for i in range(idx+threshold+1,nail_number)]
        result = left + right
    elif (idx<=threshold):
        result = [i for i in range(idx+threshold+1,nail_number-threshold+idx)]
    elif (idx>=nail_number-threshold):
        result = [i for i in range(threshold,idx-threshold)]
    return result   

In [ ]:
number = 50
threshold = 1
res = 255*np.ones(shape=(gray.shape[0],gray.shape[1]))
line_dict = {}
nail_positions = generate_nail_positions(center, radius, number)

In [ ]:
current_idx = 0
for n in range(20):
    print(n)
    m = math.inf
    possible_next_ways = possible_next_nails(current_idx, len(nail_positions),threshold)
    #start_time = time.time()
    for i in possible_next_ways:     
        tmp_res = add_line(current_idx,i,nail_positions,res,line_dict)        
        tmp_m = distance_image(tmp_res,gray_crop)
        if tmp_m < m:
            m = tmp_m
            min_res = tmp_res
            next_position = i
    res = min_res
    current_idx = next_position
    #print("--- add_line2 %s seconds ---" % (time.time() - start_time))
    #print("distance: "+str(m))
plot_gray(res)

In [ ]:
plot_gray(res)

test

In [ ]:
number = 400
threshold = 1
nail_positions = generate_nail_positions(center, radius, number)

In [ ]:
test_image = 255*np.ones(shape=(gray.shape[0],gray.shape[1]))
plot_gray(test_image)

In [ ]:
test_image

In [ ]:
number = 10
threshold = 1
nail_positions = generate_nail_positions(center, radius, number)

In [ ]:
nail_positions[20]

In [ ]:
line = [nail_positions[20],nail_positions[359]]
line_matrix = generate_line_matrix(line,test_image.shape)

In [ ]:
line_matrix[562][653]

In [ ]:
plot_gray(line_matrix)

In [ ]:
plot_gray(add_line(20,359,nail_positions,test_image,{}))

In [ ]:
test_image = 255*np.ones(shape=(gray.shape[0],gray.shape[1]))
for i in range(9):
    test_image = add_line(i,i+1,nail_positions,test_image,{})
plot_gray(test_image)

In [ ]:
test_image = 255*np.ones(shape=(gray.shape[0],gray.shape[1]))
test_image = add_line(0,1,nail_positions,test_image,{})
test_image = add_line(0,7,nail_positions,test_image,{})
plot_gray(test_image)

In [ ]:
nail_positions

In [ ]:
test_image = 255*np.ones(shape=(gray.shape[0],gray.shape[1]))
for j in range(10):
    for i in range(9):
        test_image = add_line(i,i+1,nail_positions,test_image,{})
    test_image = add_line(9,0,nail_positions,test_image,{})
plot_gray(test_image)

In [ ]:
test_image

In [ ]:
test_image = 255*np.ones(shape=(gray.shape[0],gray.shape[1]))
for j in range(10):
    for i in range(9):
        test_image = add_line(i,i+1,nail_positions,test_image,{})
    test_image = add_line(9,0,nail_positions,test_image,{})
for i in range(test_image.shape[1]):
    if (i<450) or (i>500):
        continue
    for j in range(test_image.shape[0]):
        test_image[j][i]=0

In [ ]:
plot_gray(test_image)

In [ ]:
nail_positions[0]

In [ ]:
test_image = 255*np.ones(shape=(gray.shape[0],gray.shape[1]))
for i in range(test_image.shape[1]):
    if (i<450) or (i>500):
        continue
    for j in range(test_image.shape[0]):
        test_image[j][i]=0
plot_gray(test_image)

In [ ]:
test_crop = crop_image_circle(test_image,center,radius)
plot_gray(test_crop)

In [ ]:
number = 400
threshold = 1
res = 255*np.ones(shape=(gray.shape[0],gray.shape[1]))
line_dict = {}
nail_positions = generate_nail_positions(center, radius, number)
current_idx = 40

In [ ]:
for n in range(1):
    #print(n)
    m = math.inf
    possible_next_ways = possible_next_nails(current_idx, len(nail_positions),threshold)
    #start_time = time.time()
    for i in possible_next_ways:     
        tmp_res = add_line(current_idx,i,nail_positions,res,{})        
        tmp_m = distance_image(tmp_res,test_crop)
        if tmp_m < m:
            m = tmp_m
            min_res = tmp_res
            next_position = i
    res = min_res
    print(str(current_idx)+'->'+str(next_position))
    current_idx = next_position
    
    #print("--- add_line2 %s seconds ---" % (time.time() - start_time))
    print("distance: "+str(m))
plot_gray(res)

In [ ]:
nail_positions[40]

In [ ]:
nail_positions[396]